M. Oka, M. Fujimoto, T.K.M. Nakamura, I. Shinohara, and K.-I. Nishikawa, Phys. Rev. Lett. 101, 205004 (2008) \
Magnetic Reconnection by a Self Retreating X Line

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import cupy as cp
from scipy import stats
import matplotlib.pyplot as plt
import cv2

In [3]:
def get_rho(q_list, x, n_x, n_y, dx, dy):

    x_index = cp.floor(x[0, :] / dx).astype(cp.int64)
    y_index = cp.floor(x[1, :] / dy).astype(cp.int64)

    rho = cp.zeros([n_x, n_y])

    cx1 = x[0, :] / dx - x_index 
    cx2 = 1.0 - cx1
    cy1 = x[1, :] / dy - y_index  
    cy2 = 1.0 - cy1

    index_one_array = x_index * n_y + y_index

    rho[:, :] += cp.bincount(index_one_array, 
                            weights=q_list * cx2 * cy2, 
                            minlength=n_x*n_y
                            ).reshape(n_x, n_y)
    rho[:, :] += cp.roll(cp.bincount(index_one_array, 
                                    weights=q_list * cx1 * cy2, 
                                    minlength=n_x*n_y
                                    ).reshape(n_x, n_y), 1, axis=0)
    rho[:, :] += cp.roll(cp.bincount(index_one_array, 
                                    weights=q_list * cx2 * cy1, 
                                    minlength=n_x*n_y
                                    ).reshape(n_x, n_y), 1, axis=1)
    rho[:, :] += cp.roll(cp.bincount(index_one_array, 
                                    weights=q_list * cx1 * cy1, 
                                    minlength=n_x*n_y
                                    ).reshape(n_x, n_y), [1, 1], axis=[0, 1])
    
    return rho


def solve_poisson_not_periodic(rho, n_x, n_y, dx, dy, epsilon0, E):

    phi = cp.zeros([n_x, n_y])

    for k in range(10000):
        phi = phi + (((cp.roll(phi, -1, axis=0) + cp.roll(phi, 1, axis=0))/dx**2
               +(cp.roll(phi, -1, axis=1) + cp.roll(phi, 1, axis=1))/dy**2
               + rho/epsilon0) / (2 * (1/dx**2 + 1/dy**2)) - phi) * 1.0

    E[0] = -(cp.roll(phi, -1, axis=0) - phi) / dx
    E[1] = -(cp.roll(phi, -1, axis=1) - phi) / dy

    return E



def E_modification(q_list, x, n_x, n_y, dx, dy, epsilon0, E):

    rho = get_rho(q_list, x, n_x, n_y, dx, dy)
    div_E = (E[0, :, :] - cp.roll(E[0, :, :], 1, axis=0)) / dx \
          + (E[1, :, :] - cp.roll(E[1, :, :], 1, axis=1)) / dy 
    delta_rho = rho - div_E

    delta_E = cp.zeros(E.shape)
    delta_E = solve_poisson_not_periodic(delta_rho, n_x, n_y, dx, dy, epsilon0, delta_E)

    E += delta_E
    
    return E 


def current_component(current_comp, cx1, cx2, cy1, cy2, 
                      gamma, index_one_array, n_x, n_y, 
                      q_list, v_comp):

    current_comp += cp.bincount(index_one_array, 
                                weights=q_list * v_comp/gamma * cx2 * cy2, 
                                minlength=n_x*n_y
                                ).reshape(n_x, n_y)
    current_comp += cp.roll(cp.bincount(index_one_array, 
                                        weights=q_list * v_comp/gamma * cx1 * cy2, 
                                        minlength=n_x*n_y
                                        ).reshape(n_x, n_y), 1, axis=0)
    current_comp += cp.roll(cp.bincount(index_one_array, 
                                        weights=q_list * v_comp/gamma * cx2 * cy1, 
                                        minlength=n_x*n_y
                                        ).reshape(n_x, n_y), 1, axis=1)
    current_comp += cp.roll(cp.bincount(index_one_array, 
                                        weights=q_list * v_comp/gamma * cx1 * cy1, 
                                        minlength=n_x*n_y
                                        ).reshape(n_x, n_y), [1, 1], axis=[0, 1])
    
    return current_comp


def get_current_density(c, q_list, v, x, n_x, n_y, dx, dy, current):

    x_index = cp.floor(x[0, :] / dx).astype(int)
    y_index = cp.floor(x[1, :] / dy).astype(int)

    gamma = cp.sqrt(1.0 + (cp.linalg.norm(v, axis=0)/c)**2)

    cx1 = x[0, :] / dx - x_index 
    cx2 = 1.0 - cx1
    cy1 = x[1, :] / dy - y_index  
    cy2 = 1.0 - cy1

    current = cp.zeros(current.shape)

    index_one_array = x_index * n_y + y_index

    current[0, :, :] = current_component(current[0, :, :], cx1, cx2, cy1, cy2, 
                                         gamma, index_one_array, n_x, n_y, 
                                         q_list, v[0, :])
    current[1, :, :] = current_component(current[1, :, :], cx1, cx2, cy1, cy2, 
                                         gamma, index_one_array, n_x, n_y, 
                                         q_list, v[1, :])
    current[2, :, :] = current_component(current[2, :, :], cx1, cx2, cy1, cy2, 
                                         gamma, index_one_array, n_x, n_y, 
                                         q_list, v[2, :])
    
    current[0, :, :] = (current[0, :, :] + cp.roll(current[0, :, :], -1, axis=0)) / 2.0
    current[1, :, :] = (current[1, :, :] + cp.roll(current[1, :, :], -1, axis=1)) / 2.0
    
    return current


def buneman_boris_v(c, dt, q_list, m_list, E, B, v):

    gamma = cp.sqrt(1.0 + (cp.linalg.norm(v, axis=0)/c)**2)

    #TとSの設定
    T = (q_list/m_list) * dt * B / 2.0 / gamma
    S = 2.0 * T / (1.0 + cp.linalg.norm(T, axis=0)**2)

    #時間発展
    v_minus = v + (q_list/m_list) * E * (dt/2)
    v_0 = v_minus + cp.cross(v_minus, T, axis=0)
    v_plus = v_minus + cp.cross(v_0, S, axis=0)
    v = v_plus + (q_list/m_list) * E * (dt/2.0)

    return v 


def buneman_boris_x(c, dt, v, x):

    gamma = cp.sqrt(1.0 + (cp.linalg.norm(v, axis=0)/c)**2)

    x = x + v * dt / gamma

    return x


def time_evolution_v(c, E, B, x, q_list, m_list, n_x, n_y, dx, dy, dt, v):
    
    E_tmp = E.copy()
    B_tmp = B.copy()

    #整数格子点上に再定義。特に磁場は平均の取り方に注意。
    E_tmp[0, :, :] = (E[0, :, :] + cp.roll(E[0, :, :], 1, axis=0)) / 2.0
    E_tmp[1, :, :] = (E[1, :, :] + cp.roll(E[1, :, :], 1, axis=1)) / 2.0
    B_tmp[0, :, :] = (B[0, :, :] + cp.roll(B[0, :, :], 1, axis=1)) / 2.0
    B_tmp[1, :, :] = (B[1, :, :] + cp.roll(B[1, :, :], 1, axis=0)) / 2.0
    B_tmp[2, :, :] = (B[2, :, :] + cp.roll(B[2, :, :], 1, axis=0) + cp.roll(B[2, :, :], 1, axis=1) + cp.roll(B[2, :, :], [1, 1], axis=[0, 1])) / 4.0

    x_index = cp.floor(x[0, :] / dx).astype(int)
    y_index = cp.floor(x[1, :] / dy).astype(int)

    E_particle = cp.zeros(x.shape)
    B_particle = cp.zeros(x.shape)

    cx1 = x[0, :] / dx - x_index 
    cx2 = 1.0 - cx1
    cy1 = x[1, :] / dy - y_index  
    cy2 = 1.0 - cy1
    cx1 = cx1.reshape(-1, 1)
    cx2 = cx2.reshape(-1, 1)
    cy1 = cy1.reshape(-1, 1)
    cy2 = cy2.reshape(-1, 1)

    #電場
    E_particle[:, :] = (E_tmp[:, x_index, y_index].T * (cx2 * cy2) \
                     + E_tmp[:, (x_index+1)%n_x, y_index].T * (cx1 * cy2) \
                     + E_tmp[:, x_index, (y_index+1)%n_y].T * (cx2 * cy1) \
                     + E_tmp[:, (x_index+1)%n_x, (y_index+1)%n_y].T * (cx1 * cy1)
                    ).T
    
    #磁場
    B_particle[:, :] = (B_tmp[:, x_index, y_index].T * (cx2 * cy2) \
                     + B_tmp[:, (x_index+1)%n_x, y_index].T * (cx1 * cy2) \
                     + B_tmp[:, x_index, (y_index+1)%n_y].T * (cx2 * cy1) \
                     + B_tmp[:, (x_index+1)%n_x, (y_index+1)%n_y].T * (cx1 * cy1)
                    ).T
  
    v = buneman_boris_v(c, dt, q_list, m_list, E_particle, B_particle, v)

    return v


def time_evolution_x(c, dt, v, x):
    
    x = buneman_boris_x(c, dt, v, x)

    return x 


def time_evolution_E(B, current, c, epsilon0, dx, dy, dt, E):

    E[0, :, :] += (-current[0, :, :]/epsilon0 \
               + c**2 * (B[2, :, :] - cp.roll(B[2, :, :], 1, axis=1))/dy) * dt
    E[1, :, :] += (-current[1, :, :]/epsilon0 \
               - c**2 * (B[2, :, :] - cp.roll(B[2, :, :], 1, axis=0))/dx) * dt
    E[2, :, :] += (-current[2, :, :]/epsilon0 \
               + c**2 * ((B[1, :, :] - cp.roll(B[1, :, :], 1, axis=0))/dx \
               - (B[0, :, :] - cp.roll(B[0, :, :], 1, axis=1))/dy)) * dt
    
    return E


def time_evolution_B(E, dx, dy, dt, B):

    B[0, :, :] += -(cp.roll(E[2, :, :], -1, axis=1) - E[2, : , :])/dy * dt
    B[1, :, :] += (cp.roll(E[2, :, :], -1, axis=0) - E[2, :, :])/dx * dt
    B[2, :, :] += (-(cp.roll(E[1, :, :], -1, axis=0) - E[1, :, :])/dx \
               + (cp.roll(E[0, :, :], -1, axis=1) - E[0, :, :])/dy) * dt
    
    return B


def periodic_condition_x(x, x_max):

    over_xmax_index = cp.where(x[0, :] >= x_max)[0]
    x[0, over_xmax_index] = 1e-10

    under_x0_index = cp.where(x[0, :] <= 0.0)[0]
    x[0, under_x0_index] = x_max - 1e-10

    return x 


def periodic_condition_y(x, y_max):

    over_ymax_index = cp.where(x[1, :] >= y_max)[0]
    x[1, over_ymax_index] = 1e-10

    under_y0_index = cp.where(x[1, :] <= 0.0)[0]
    x[1, under_y0_index] = y_max - 1e-10

    return x



def refrective_condition_x(v, x, x_max):

    over_xmax_index = cp.where(x[0, :] >= x_max)[0]
    x[0, over_xmax_index] = x_max - 1e-10
    v[0, over_xmax_index] = -v[0, over_xmax_index]

    under_x0_index = cp.where(x[0, :] <= 0.0)[0]
    x[0, under_x0_index] = 1e-10
    v[0, under_x0_index] = -v[0, under_x0_index]

    return v, x


def refrective_condition_y(v, x, y_max):

    over_ymax_index = cp.where(x[1, :] >= y_max)[0]
    x[1, over_ymax_index] = y_max - 1e-10
    v[1, over_ymax_index] = -v[1, over_ymax_index]
    
    under_y0_index = cp.where(x[1, :] <= 0.0)[0]
    x[1, under_y0_index] = 1e-10
    v[1, under_y0_index] = -v[1, under_y0_index]

    return v, x


## 全過程

初期設定

In [5]:
c = 0.5
epsilon0 = 1.0
mu_0 = 1.0 / (epsilon0 * c**2)
m_unit = 1.0
r_m = 1/9
m_electron = 1 * m_unit
m_ion = m_electron / r_m
t_r = 1.0
r_q = 1.0
n_e = 10 #ここは手動で調整すること
B0 = np.sqrt(n_e) / 1.5
n_i = int(n_e / r_q)
T_i  = (B0**2 / 2.0 / mu_0) / (n_i + n_e * t_r)
T_e = T_i * t_r
q_unit = np.sqrt(epsilon0 * T_e / n_e)
q_electron = -1 * q_unit
q_ion = r_q * q_unit
debye_length = np.sqrt(epsilon0 * T_e / n_e / q_electron**2)
omega_pe = np.sqrt(n_e * q_electron**2 / m_electron / epsilon0)
omega_pi = np.sqrt(n_i * q_ion**2 / m_ion / epsilon0)
omega_ce = q_electron * B0 / m_electron
omega_ci = q_ion * B0 / m_ion
ion_inertial_length = c / omega_pi
sheat_thickness = 1.0 * ion_inertial_length
v_electron = np.array([0.0, 0.0, c * debye_length / sheat_thickness * np.sqrt(2 / (1.0 + 1/t_r))])
v_ion = -v_electron / t_r
v_thermal_electron = np.sqrt(T_e / m_electron)
v_thermal_ion = np.sqrt(T_i / m_ion)

dx = 1.0
dy = 1.0
n_x = int(ion_inertial_length * 200)
n_y = int(ion_inertial_length * 50)
x_max = n_x * dx
y_max = n_y * dy
x_coordinate = cp.arange(0.0, x_max, dx)
y_coordinate = cp.arange(0.0, y_max, dy)
dt = 1.0
step = 20000
t_max = step * dt


E = cp.zeros([3, n_x, n_y])
B = cp.zeros([3, n_x, n_y])
current = cp.zeros([3, n_x, n_y])
for j in range(n_y):
    B[0, :, j] = B0 * cp.tanh((y_coordinate[j] - y_max/2) / sheat_thickness)

B[2, :, :] = 0.6 * B0

reconnection_ratio = 0.1
Xpoint_position = 10 * ion_inertial_length
delta_B = cp.zeros([3, n_x, n_y])
X, Y = cp.meshgrid(x_coordinate, y_coordinate)
delta_B[0, :, :] = -cp.array(reconnection_ratio * B0 * (Y - y_max/2) / sheat_thickness \
                 * cp.exp(-((X - Xpoint_position)**2 + (Y - y_max/2)**2) / ((2.0 * sheat_thickness)**2))).T
delta_B[1, :, :] = cp.array(reconnection_ratio * B0 * (X - Xpoint_position) / sheat_thickness \
                 * cp.exp(-((X - Xpoint_position)**2 + (Y - y_max/2)**2) / ((2.0 * sheat_thickness)**2))).T

n_ion = int(n_x * n_i * 2.0 * sheat_thickness)
n_electron = int(n_ion * abs(q_ion / q_electron))
n_ion_background = int(n_x * 0.2 * n_i * (y_max - 2.0 * sheat_thickness))
n_electron_background = int(n_x * 0.2 * n_e * (y_max - 2.0 * sheat_thickness))
x = cp.zeros([3, n_ion + n_ion_background + n_electron + n_electron_background])
v = cp.zeros([3, n_ion + n_ion_background + n_electron + n_electron_background])
print(f"total number of particles is {n_ion + n_ion_background + n_electron + n_electron_background}.")

np.random.RandomState(1)
x_start_plus = np.random.rand(n_ion) * x_max
x_start_plus_background = np.random.rand(n_ion_background) * x_max
x_start_minus = np.random.rand(n_electron) * x_max
x_start_minus_background = np.random.rand(n_electron_background) * x_max
y_start_plus = np.array(y_max/2 + sheat_thickness * np.arctanh(2.0 * np.random.rand(n_ion) - 1.0))
y_start_plus[y_start_plus > y_max] = y_max/2
y_start_plus[y_start_plus < 0.0] = y_max/2
y_start_plus_background = np.zeros(n_ion_background)
for i in range(n_ion_background):
    while True:
        rand = np.random.rand(1) * y_max 
        rand_pn = np.random.rand(1)
        if rand_pn < (1.0 - 1.0/np.cosh((rand - y_max/2)/sheat_thickness)):
            y_start_plus_background[i] = rand
            break
y_start_minus = np.array(y_max/2 + sheat_thickness * np.arctanh(2.0 * np.random.rand(n_electron) - 1.0))
y_start_minus[y_start_minus > y_max] = y_max/2
y_start_minus[y_start_minus < 0.0] = y_max/2
y_start_minus_background = np.zeros(n_electron_background)
for i in range(n_electron_background):
    while True:
        rand = np.random.rand(1) * y_max 
        rand_pn = np.random.rand(1)
        if rand_pn < (1.0 - 1.0/np.cosh((rand - y_max/2)/sheat_thickness)):
            y_start_minus_background[i] = rand
            break
x_start_plus = cp.asarray(x_start_plus)
x_start_plus_background = cp.asarray(x_start_plus_background)
x_start_minus = cp.asarray(x_start_minus)
x_start_minus_background = cp.asarray(x_start_minus_background)
y_start_plus = cp.asarray(y_start_plus)
y_start_plus_background = cp.asarray(y_start_plus_background)
y_start_minus = cp.asarray(y_start_minus)
y_start_minus_background = cp.asarray(y_start_minus_background)
x[0, :] = cp.concatenate([x_start_plus, x_start_plus_background, x_start_minus, x_start_minus_background])
x[1, :] = cp.concatenate([y_start_plus, y_start_plus_background, y_start_minus, y_start_minus_background])
v[0, :n_ion] = cp.asarray(stats.norm.rvs(v_ion[0], v_thermal_ion, size=n_ion))
v[0, n_ion:n_ion+n_ion_background] = cp.asarray(stats.norm.rvs(0.0, v_thermal_ion, size=n_ion_background))
v[0, n_ion+n_ion_background:n_ion+n_ion_background+n_electron] = cp.asarray(stats.norm.rvs(v_electron[0], v_thermal_electron, size=n_electron))
v[0, n_ion+n_ion_background+n_electron:] = cp.asarray(stats.norm.rvs(0.0, v_thermal_electron, size=n_electron_background))
v[1, :n_ion] = cp.asarray(stats.norm.rvs(v_ion[1], v_thermal_ion, size=n_ion))
v[1, n_ion:n_ion+n_ion_background] = cp.asarray(stats.norm.rvs(0.0, v_thermal_ion, size=n_ion_background))
v[1, n_ion+n_ion_background:n_ion+n_ion_background+n_electron] = cp.asarray(stats.norm.rvs(v_electron[1], v_thermal_electron, size=n_electron))
v[1, n_ion+n_ion_background+n_electron:] = cp.asarray(stats.norm.rvs(0.0, v_thermal_electron, size=n_electron_background))
v[2, :n_ion] = cp.asarray(stats.norm.rvs(v_ion[2], v_thermal_ion, size=n_ion))
v[2, n_ion:n_ion+n_ion_background] = cp.asarray(stats.norm.rvs(0.0, v_thermal_ion, size=n_ion_background))
v[2, n_ion+n_ion_background:n_ion+n_ion_background+n_electron] = cp.asarray(stats.norm.rvs(v_electron[2], v_thermal_electron, size=n_electron))
v[2, n_ion+n_ion_background+n_electron:] = cp.asarray(stats.norm.rvs(0.0, v_thermal_electron, size=n_electron_background))

q_list = cp.zeros(n_ion + n_ion_background + n_electron + n_electron_background)
q_list[:n_ion+n_ion_background] = q_ion
q_list[n_ion+n_ion_background:] = q_electron
m_list = cp.zeros(n_ion + n_ion_background + n_electron + n_electron_background)
m_list[:n_ion+n_ion_background] = m_ion
m_list[n_ion+n_ion_background:] = m_electron

total number of particles is 18560000.


KeyboardInterrupt: ignored

In [ ]:
#STEP1
rho = get_rho(q_list, x, n_x, n_y, dx, dy)
E = solve_poisson_not_periodic(rho, n_x, n_y, dx, dy, epsilon0, E)  

for k in range(step+1):
    if k == 0:
        B += delta_B
    #STEP2
    B = time_evolution_B(E, dx, dy, dt/2, B) 
    B[1, :, [0, 1]] = 0.0
    B[1, :, -1] = 0.0
    B[0, :, 0] = -mu_0 * current[2, :, 0]
    B[2, :, 0] = mu_0 * current[0, :, 0]
    B[0, :, -1] = mu_0 * current[2, :, -1]
    B[2, :, -1] = -mu_0 * current[0, :, -1]
    B[1, 0, :] = 0.0
    B[1, [-1, -2], :] = 0.0
    B[2, 0, :] = 0.0
    B[2, [-1, -2], :] = 0.0
    B[0, [0, 1], :] = B[0, [1, 2], :]
    B[0, [-1, -2], :] = B[0, [-2, -3], :]
    #STEP3
    v = time_evolution_v(c, E, B, x, q_list, m_list, n_x, n_y, dx, dy, dt, v)
    #STEP4
    x = time_evolution_x(c, dt/2, v, x)
    v, x = refrective_condition_x(v, x, x_max)
    v, x = refrective_condition_y(v, x, y_max)
    #STEP5
    current = get_current_density(c, q_list, v, x, n_x, n_y, dx, dy, current)
    #STEP6
    B = time_evolution_B(E, dx, dy, dt/2, B) 
    B[1, :, [0, 1]] = 0.0
    B[1, :, -1] = 0.0
    B[0, :, 0] = -mu_0 * current[2, :, 0]
    B[2, :, 0] = mu_0 * current[0, :, 0]
    B[0, :, -1] = mu_0 * current[2, :, -1]
    B[2, :, -1] = -mu_0 * current[0, :, -1]
    B[1, 0, :] = 0.0
    B[1, [-1, -2], :] = 0.0
    B[2, 0, :] = 0.0
    B[2, [-1, -2], :] = 0.0
    B[0, [0, 1], :] = B[0, [1, 2], :]
    B[0, [-1, -2], :] = B[0, [-2, -3], :]
    #STEP7
    if k % 100 == 0:
        print(f'{int(k*dt)} step done...')
        #E = E_modification(q_list, x, n_x, n_y, dx, dy, epsilon0, E)
    E = time_evolution_E(B, current, c, epsilon0, dx, dy, dt, E)
    rho = get_rho(q_list, x, n_x, n_y, dx, dy)
    E[1, :, 0] = rho[:, 0] / epsilon0
    E[1, :, -1] = -rho[:, -1] / epsilon0
    E[0, :, [0, 1]] = 0.0
    E[2, :, [0, 1]] = 0.0
    E[0, :, -1] = 0.0
    E[2, :, -1] = 0.0
    E[0, 0, :] = 0.0
    E[0, [-1, -2], :] = 0.0
    E[1, [0, 1], :] = E[1, [1, 2], :]
    E[1, [-1, -2], :] = E[1, [-2, -3], :]
    E[2, [0, 1], :] = E[2, [1, 2], :]
    E[2, [-1, -2], :] = E[2, [-2, -3], :]
    #STEP8
    x = time_evolution_x(c, dt/2, v, x)
    v, x = refrective_condition_x(v, x, x_max)
    v, x = refrective_condition_y(v, x, y_max)

    if k % 100 == 0:
        k1 = k // 100
        KE = np.sum(1/2 * m_list * np.linalg.norm(v, axis=0)**2)
        np.save(f'results_mr2008_xv_{k1}.npy', np.concatenate([x, v]))
        np.save(f'results_mr2008_E_{k1}.npy', E)
        np.save(f'results_mr2008_B_{k1}.npy', B)
        np.save(f'results_mr2008_current_{k1}.npy', current)
        np.save(f'results_mr2008_KE_{k1}.npy', KE)

        import locale
        def getpreferredencoding(do_setlocale = True):
            return "UTF-8"
        locale.getpreferredencoding = getpreferredencoding
        !mv 'results_mr2008_xv_{k1}.npy' './drive/MyDrive/Colab Notebooks/mr2008_test/'
        !mv 'results_mr2008_E_{k1}.npy' './drive/MyDrive/Colab Notebooks/mr2008_test/'
        !mv 'results_mr2008_B_{k1}.npy' './drive/MyDrive/Colab Notebooks/mr2008_test/'
        !mv 'results_mr2008_current_{k1}.npy' './drive/MyDrive/Colab Notebooks/mr2008_test/'
        !mv 'results_mr2008_KE_{k1}.npy' './drive/MyDrive/Colab Notebooks/mr2008_test/'

0 step done...
100 step done...
200 step done...
300 step done...
400 step done...
500 step done...
600 step done...
700 step done...
800 step done...
900 step done...
1000 step done...
1100 step done...
1200 step done...
1300 step done...
1400 step done...
1500 step done...
1600 step done...
1700 step done...
1800 step done...
1900 step done...
2000 step done...
2100 step done...
2200 step done...
2300 step done...
2400 step done...
2500 step done...
2600 step done...
2700 step done...
2800 step done...
2900 step done...
3000 step done...
3100 step done...
3200 step done...
3300 step done...
